In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt


# Device Configuration

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#Data

In [ ]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import random_split


transform = transforms.ToTensor()

train_data = datasets.CIFAR10(
    root='./data/',
    train=True,
    download=True,
    transform=transform
)
test_data=datasets.CIFAR10(
    root='./data/',
    train=False,
    download=True,
    transform=transform
)
train_set, val_set = random_split( train_data,[40000, 10000])

train_loader=torch.utils.data.DataLoader(train_set,batch_size=128,shuffle=True)
validation_loader=torch.utils.data.DataLoader(val_set,batch_size=128,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=128,shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 3, 32, 32])
Image label dimensions: torch.Size([128])


# Model

In [ ]:
num_classes = 10
class VGG19(torch.nn.Module):

    def __init__(self, num_classes):
        super(VGG19, self).__init__()


        self.block_1 = nn.Sequential(
                nn.Conv2d(in_channels=3,
                          out_channels=64,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=64,
                          out_channels=64,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )

        self.block_2 = nn.Sequential(
                nn.Conv2d(in_channels=64,
                          out_channels=128,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=128,
                          out_channels=128,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )

        self.block_3 = nn.Sequential(
                nn.Conv2d(in_channels=128,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )


        self.block_4 = nn.Sequential(
                nn.Conv2d(in_channels=256,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )

        self.block_5 = nn.Sequential(
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )

        self.classifier = nn.Sequential(
                nn.Linear(512, 4096),
                nn.ReLU(True),
                nn.Linear(4096, 4096),
                nn.ReLU(True),
                nn.Linear(4096, num_classes)
        )

        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                m.weight.detach().normal_(0, 0.05)
                if m.bias is not None:
                    m.bias.detach().zero_()
            elif isinstance(m, torch.nn.Linear):
                m.weight.detach().normal_(0, 0.05)
                m.bias.detach().detach().zero_()



    def forward(self, x):

        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.block_5(x)
        x = self.classifier(x.view(-1, 512))

        return x

model = VGG19(num_classes=num_classes)
model = model.to(device)



# Defining  Training Loop

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn):
    for epoch in range(n_epochs):

      train_loss=0.0
      model.train()
      for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

      val_loss=0.0
      model.eval()
      for inputs, labels in validation_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        val_loss += loss.item()

      print("Epoch: {} /{} Train Loss: {} Val Loss: {}".format(epoch+1,n_epochs, train_loss/len(train_loader), val_loss/len(validation_loader)))



# Training


In [ ]:
optimizer=optim.Adam(model.parameters(),lr=0.001)
loss_fn=nn.CrossEntropyLoss()
training_loop(
    n_epochs=10,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn)



Epoch: 1 /10 Train Loss: 26.9989597584112 Val Loss: 1.7366294694852225
Epoch: 2 /10 Train Loss: 1.609852732560886 Val Loss: 1.4814985356753385
Epoch: 3 /10 Train Loss: 1.3717634700738583 Val Loss: 1.3256483100637604
Epoch: 4 /10 Train Loss: 1.2250712862410866 Val Loss: 1.1693562208851682
Epoch: 5 /10 Train Loss: 1.0967882774508417 Val Loss: 1.0798174105113065
Epoch: 6 /10 Train Loss: 1.0088369459770739 Val Loss: 0.9992229878147946
Epoch: 7 /10 Train Loss: 0.9313554535277735 Val Loss: 1.0243790889088111
Epoch: 8 /10 Train Loss: 0.8773880585694847 Val Loss: 0.9714316061780423
Epoch: 9 /10 Train Loss: 0.8013078915044523 Val Loss: 0.9139565765103207
Epoch: 10 /10 Train Loss: 0.7359194137608281 Val Loss: 0.8808421832096728


In [ ]:
num_correct = 0.0
for x_test_batch, y_test_batch in test_loader:
  model.eval()
  y_test_batch = y_test_batch.to(device)
  x_test_batch = x_test_batch.to(device)
  y_pred_batch = model(x_test_batch)
  _, predicted = torch.max(y_pred_batch, 1)
  num_correct += (predicted ==y_test_batch).float().sum()
  accuracy = num_correct/(len(test_loader) *test_loader.batch_size)
print(len(test_loader), test_loader.batch_size)
print("Test Accuracy: {}".format(accuracy))

79 128
Test Accuracy: 0.6847310066223145
